In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
from datetime import datetime
import re
from scipy.stats.stats import pearsonr


In [3]:
df = pd.read_csv("2014_data.csv",low_memory=False,skiprows=1)

In [4]:
df15 = pd.read_csv("K:/DI/2015_data.csv",low_memory=False,skiprows=1)

In [20]:
#df["loan_amnt"].value_counts()
df15.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_il_6m,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog
0,NaN,NaN,3600.0,3600.0,3600.0,36 months,13.99%,123.03,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,35000.0,35000.0,35000.0,60 months,14.85%,829.90,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,29900.0,29900.0,29900.0,60 months,12.88%,678.49,C,C2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,18000.0,18000.0,18000.0,60 months,11.99%,400.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
all_data = pd.concat([df,df15])

In [6]:
#checked
print (len(all_data['loan_amnt']))
print (len(df['loan_amnt']))
print(len(df15['loan_amnt']))

656728
235631
421097


In [26]:
all_data['loan_amnt'].median()

member_id                                     NaN
loan_amnt                               13750.000
funded_amnt                             13750.000
funded_amnt_inv                         13750.000
installment                               385.110
annual_inc                              65000.000
url                                           NaN
dti                                        18.230
delinq_2yrs                                 0.000
inq_last_6mths                              0.000
mths_since_last_delinq                     30.000
mths_since_last_record                     67.000
open_acc                                   11.000
pub_rec                                     0.000
revol_bal                               11886.500
total_acc                                  24.000
out_prncp                                   0.000
out_prncp_inv                               0.000
total_pymnt                             10583.775
total_pymnt_inv                         10579.540


In [7]:
#problem 1
all_data['loan_amnt'].median()

13750.0

In [8]:
#problem 2
pur = []
loan = []
for p,l in zip(all_data['purpose'],all_data['loan_amnt']):
    if p in pur:
        loan[pur.index(p)].append(l)
    else:
        pur.append(p)
        loan.append([])
        loan[-1].append(l)

loan_mean = []
for i in range(len(pur)):
    if str(np.mean(loan[i])) == 'nan':
        continue
    loan_mean.append(np.mean(loan[i]))
    print(pur[i],np.mean(loan[i]))
print(np.amax(loan_mean))
print(np.amin(loan_mean))
print (np.amin(loan_mean)/np.amax(loan_mean))

credit_card 15747.3409522
debt_consolidation 15630.4946747
car 9765.17081918
home_improvement 14679.8372372
house 14426.416819
other 10273.6204565
medical 9223.75179454
moving 8143.64994664
major_purchase 12774.9933669
vacation 6477.07907791
small_business 15732.3967382
renewable_energy 10084.7982709
wedding 11204.1666667
educational 2200.0
15747.3409522
2200.0
0.139706126048


In [192]:
#Problem 3
#check status
#all_data['loan_status'].value_counts()
#all_data['term'].value_counts()
#all_data['last_pymnt_d'].value_counts()
#all_data['issue_d'].value_counts()
#all_data['loan_status'].value_counts()

"""last_pymnt_d - issue_d   (in month-year format)/ term of loan(in month, 36/60),
for status not in 3 mentioned"""

ratio = []
a3 = (all_data['loan_status'] != "Current") 
b3 = (all_data['loan_status'] != "Fully Paid") 
c3 =(all_data['loan_status'] != "In Grace Period")
d3 = all_data[a3 & b3 & c3]

def time_spent(lst, isu):
    lst = datetime.strptime(lst, "%b-%Y")
    isu = datetime.strptime(isu, "%b-%Y")
    return (lst-isu).days

for lst, isu,ter in zip(d3['last_pymnt_d'],d3['issue_d'], d3['term']):
    try:
        if str(lst) == 'nan' :
            continue
        term_days = float(str.split(ter)[0])*365.0/12
        ratio.append(time_spent(lst,isu)/term_days)
    except TypeError:
        print(lst)
        print(lst,isu,ter)

    #
print (np.mean(ratio))
print (np.std(ratio))

0.345212791706
0.197064471818


In [37]:
#all_data['term'].head()
#all_data['term'].value_counts()
#all_data['last_pymnt_d'].value_counts()
#all_data['issue_d'].value_counts()
#
#for p,l,k in zip(all_data['purpose'],all_data['loan_amnt'],all_data['term']):
#    print(p,l,k, "\n")
a = (all_data['term'] != "Current") 
b = (all_data['term'] != "Fully Paid") 
c =(all_data['term'] != "In Grace Period")
d = all_data[a & b & c]

def time_spent(lst, isu):
    lst = datetime.strptime(lst, "%m-%Y")
    isu = datetime.strptime(isu, "%m-%Y")
    return (isu-lst).month
print(d['issue_d'].value_counts())
d['term'].value_counts()
#same number

Oct-2015    48631
Jul-2015    45962
Dec-2015    44343
Oct-2014    38783
Nov-2015    37530
Aug-2015    35886
Apr-2015    35427
Jan-2015    35107
May-2015    31913
Jul-2014    29306
Sep-2015    28641
Jun-2015    28485
Mar-2015    25400
Nov-2014    25054
Feb-2015    23770
May-2014    19099
Apr-2014    19071
Aug-2014    18814
Jun-2014    17179
Mar-2014    16513
Jan-2014    15628
Feb-2014    15269
Sep-2014    10606
Dec-2014    10307
Name: issue_d, dtype: int64


 36 months    445743
 60 months    210981
Name: term, dtype: int64

In [49]:
all_data['term'].value_counts()

 36 months    445743
 60 months    210981
Name: term, dtype: int64

In [28]:
a[0]

TypeError: 'map' object is not subscriptable

In [193]:
#problem 4
"""National purpose probability
    state-purpose probability
    ratio state/nation"""

"""key to use
   addr_state
   purpose
 """

nation_prob_list = all_data['purpose'].value_counts()/float(np.sum(all_data['purpose'].value_counts()))
state_list = all_data['addr_state'].value_counts()

p4_state_purpose = []
p4_ratio = []

for state in state_list.keys():
    a = (all_data['addr_state'] == state)
    state_data = all_data[a]
    state_total_purpose_number = float(np.sum(state_data['purpose'].value_counts()))
    if state_total_purpose_number != state_list[state]:
        print(state,state_total_purpose_number,state_list[state])
    state_prob_list = state_data['purpose'].value_counts()/state_total_purpose_number
    for state_purpose in state_prob_list.keys():
        if state_prob_list[state_purpose] > 10/state_total_purpose_number:
            ratio = state_prob_list[state_purpose]/nation_prob_list[state_purpose]
            p4_state_purpose.append(state+"  "+state_purpose )
            p4_ratio.append(ratio)
print(np.amax(p4_ratio))
#print (p4_ratio)
#i = p4_ratio.index(np.amax(p4_ratio))
#print(p4_state_purpose[i])
#for i in range(len(p4_ratio)):
#    print (p4_ratio[i],p4_state_purpose[i])


#temp = all_data['addr_state'].value_counts()
#for a in temp.keys():
#    print(a)
#State probability


3.27276930868


In [105]:
state_list = all_data['addr_state'].value_counts()
for state in state_list.keys():
    print(state)

CA
NY
TX
FL
IL
NJ
PA
OH
GA
VA
NC
MI
MD
AZ
MA
WA
CO
MN
IN
TN
MO
CT
NV
WI
AL
SC
LA
OR
KY
OK
KS
AR
UT
MS
NM
WV
HI
NH
RI
DE
MT
DC
AK
WY
VT
SD
NE
ME
ND
IA
ID


In [112]:
#nation_prob_list['debt_consolidation']
state_list['CA']

91355

In [77]:
a = all_data['purpose'].value_counts().keys()

In [100]:
b = all_data['addr_state'].value_counts()
b.keys()[0]
a = (all_data['addr_state'] == b.keys()[0])
#a = (all_data['term'] != "Current") 
d= all_data[a]
d['addr_state'].value_counts()

CA    91355
Name: addr_state, dtype: int64

In [80]:
b = all_data['purpose'].value_counts()/float(np.sum(all_data['purpose'].value_counts()))

In [82]:
b.keys()

Index(['debt_consolidation', 'credit_card', 'home_improvement', 'other',
       'major_purchase', 'medical', 'small_business', 'car', 'moving',
       'vacation', 'house', 'renewable_energy', 'wedding', 'educational'],
      dtype='object')

In [194]:
#problem 5
b = all_data['purpose'].value_counts()/float(np.sum(all_data['purpose'].value_counts()))

b[b.keys()[0]]

0.59846449954623249

In [122]:
all_data['purpose'].value_counts()

debt_consolidation    393026
credit_card           157547
home_improvement       38338
other                  29575
major_purchase         11307
medical                 6269
small_business          5641
car                     5298
moving                  3748
vacation                3427
house                   2188
renewable_energy         347
wedding                   12
educational                1
Name: purpose, dtype: int64

In [195]:
#problem 6
p6_14 = df['term'].value_counts()/float(np.sum(df['term'].value_counts()))
p6_15 = df15['term'].value_counts()/float(np.sum(df15['term'].value_counts()))
p6_14.keys()
p6_14[' 36 months'] - p6_15[' 36 months']


0.017472334236841358

In [6]:
#problem 7
"""

condition reach term:
status fully paid


keys:int_rate
total_pymnt
loan_amnt

"""
interest_rate = []
total_rate = []
a = (all_data['loan_status'] == "Fully Paid") 
p7_data = all_data[a]
for tot_pmt, loa_amt,intst in zip(p7_data['total_pymnt'],p7_data['loan_amnt'], p7_data['int_rate']):
    interest_rate.append(0.01*float(intst.strip("%")))
    total_rate.append((tot_pmt-loa_amt)/loa_amt)
print(pearsonr(total_rate,interest_rate))
#print(len(total_rate))
#for i in range(len(total_rate)):
#    print (total_rate,interest_rate)
#print (np.mean(interest_rate))
#print (np.mean(total_rate))

(0.54726352361989916, 0.0)
257788


In [138]:
for lst, isu,ter in zip(all_data['last_pymnt_d'],all_data['issue_d'], all_data['term']):
    try:
        if str(lst) == 'nan' :
            continue
        term_days = float(str.split(ter)[0])*365.0/12
        if time_spent(lst,isu)/term_days>1.0:
            print (time_spent(lst,isu)/term_days)
        #ratio.append(time_spent(lst,isu)/term_days)
    except TypeError:
        print(lst)
        print(lst,isu,ter)

1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420

1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420

1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420

1.0292237442922374
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0292237442922374
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0292237442922374
1.0292237442922374
1.0292237442922374
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0292237442922374
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420

1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0292237442922374
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420

...

1.0264840182648403
1.0264840182648403
1.0547945205479452
1.0009132420091325
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0009132420091325
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0547945205479452
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0009132420091325
1.0547945205479452
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0009132420091325
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0547945205479452
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0009132420091325
1.0264840182648403
1.0009132420091325
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0547945205479452
1.0264840182648403
1.0264840182648403
1.0264840182648403
1.0264840182

1.0264840182648403
1.0264840182648403
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0264840182648403
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0264840182648403
1.0009132420091325
1.0009132420091325
1.0264840182648403
1.0009132420091325
1.082191780821918
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0264840182648403
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0264840182648403
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0264840182648403
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.02648401826

1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0264840182648403
1.0009132420091325
1.0292237442922374
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0292237442922374
1.0292237442922374
1.0292237442922374
1.0292237442922374
1.0264840182648403
1.0009132420091325
1.0292237442922374
1.0292237442922374
1.0292237442922374
1.0009132420091325
1.0292237442922374
1.0292237442922374
1.0292237442922374
1.0009132420091325
1.0292237442922374
1.0292237442922374
1.0292237442922374
1.0292237442922374
1.0547945205479452
1.0292237442922374
1.0292237442922374
1.0264840182648403
1.0292237442922374
1.0292237442922374
1.0264840182648403
1.0292237442922374
1.0292237442922374
1.0292237442922374
1.0292237442922374
1.0831050228310501
1.0292237442922374
1.0292237442922374
1.0292237442922374
1.0009132420091325
1.0292237442922374
1.0292237442922374
1.0292237442922374
1.0292237442922374
1.0292237442922374
1.0292237442922374
1.0292237442922374
1.0292237442

1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420

1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0547945205479452
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0292237442922374
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0547945205479452
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420091325
1.0009132420

In [198]:
a=[1,3,5,7]
b=[3,5,7,9]
pearsonr(a,b)

(1.0, 0.0)

In [191]:
#problem 8
"""
keys:sub_grade
int_rate
using d3 from problem4 as default
"""
a3 = (all_data['loan_status'] != "Current") 
b3 = (all_data['loan_status'] != "Fully Paid") 
c3 =(all_data['loan_status'] != "In Grace Period")
d8 = all_data[a3 & b3 & c3]

all_grade_list = all_data['sub_grade'].value_counts()
default_grade_list = d8['sub_grade'].value_counts()
#same length

grade_list = []
ave_int = []
ave_def = []

for subgrade in all_grade_list.keys():
    grade_list.append(subgrade)
    #a = (d8['sub_grade'] == subgrade)
    #grade_data = d8[a]
    a = (all_data['sub_grade'] == subgrade)
    grade_data = all_data[a]
    grade_int = []
    for int_rat in grade_data['int_rate']:
        grade_int.append(0.01*float(int_rat.strip("%")))
    ave_int.append(np.mean(grade_int))
    ave_def.append(float(default_grade_list[subgrade])/all_grade_list[subgrade])
slope,intercept = np.polyfit(ave_int, ave_def, 1)
#print(slope,intercept)
dev_list = []
for i in range(len(grade_list)):
    dev_list.append(abs(slope*ave_int[i] + intercept - ave_def[i]))
print(np.amax(dev_list))

0.0304874826351


In [190]:
# d8['sub_grade'].value_counts()
# a = (d8['sub_grade'] == "C4")
# grade_data = d8[a]
# grade_data["int_rate"]
#a = (all_data['sub_grade'] == "C4")
#grade_data = all_data[a]
#len(all_grade_list)
#np.mean(all_grade_list)

#for a in grade_data['int_rate']:
#    print(a)
#len(grade_data)
# for i in range(len(grade_list)):
#     print (grade_list[i],ave_int[i],ave_def[i],dev_list[i])

slope,intercept = np.polyfit(ave_int, ave_def, 1)
#print(slope,intercept)
dev_list = []
for i in range(len(grade_list)):
    dev_list.append(abs(slope*ave_int[i] + intercept - ave_def[i]))
print(np.amax(dev_list))
# i = dev_list.index(np.amax(dev_list))
# print(grade_list[i],ave_int[i],ave_def[i])

C1 0.12543730592 0.122383051187 0.00910970831354
C2 0.130131580548 0.137072045817 0.00309540482627
C3 0.136401669341 0.145375019411 0.00637911672366
B4 0.113247569696 0.0939216499119 0.0150453297312
C4 0.142338493857 0.158964196063 0.00376077648959
B3 0.104416623356 0.0852513788714 0.00739662966798
B5 0.119408875034 0.110246679317 0.0101059616007
A5 0.0811066883501 0.0556771141336 0.00610423493229
B2 0.0958426896757 0.0752157096102 0.00158826924312
B1 0.0857599028503 0.064522041976 0.00635034787508
C5 0.148831849885 0.170047350909 0.00467688936754
D1 0.157228603224 0.184765050077 0.00547580300794
A4 0.0739579683056 0.046688338074 0.0103257934549
D2 0.16552226806 0.208735034303 0.0031680687511
D3 0.170958057143 0.206260869565 0.00935105397393
D4 0.176783531541 0.22373319545 0.00264379727915
A1 0.0562850742009 0.0178082191781 0.0141039466282
A2 0.0637232178011 0.0316172522818 0.0141678106642
A3 0.0699879206264 0.0359213436137 0.00689516939701
D5 0.181450065531 0.232353491855 0.0026469292

In [183]:
x=[1,2,3,4,5]
y=[1,2,3,4,5]
#regr.fit(x, y)
from sklearn import datasets, linear_model
diabetes = datasets.load_diabetes()
diabetes_X = diabetes.data[:, np.newaxis, 2]
diabetes_y_train = diabetes.target[:-20]
print(diabetes_X[1:3])
print(diabetes_y_train)

[[-0.05147406]
 [ 0.04445121]]
[ 151.   75.  141.  206.  135.   97.  138.   63.  110.  310.  101.   69.
  179.  185.  118.  171.  166.  144.   97.  168.   68.   49.   68.  245.
  184.  202.  137.   85.  131.  283.  129.   59.  341.   87.   65.  102.
  265.  276.  252.   90.  100.   55.   61.   92.  259.   53.  190.  142.
   75.  142.  155.  225.   59.  104.  182.  128.   52.   37.  170.  170.
   61.  144.   52.  128.   71.  163.  150.   97.  160.  178.   48.  270.
  202.  111.   85.   42.  170.  200.  252.  113.  143.   51.   52.  210.
   65.  141.   55.  134.   42.  111.   98.  164.   48.   96.   90.  162.
  150.  279.   92.   83.  128.  102.  302.  198.   95.   53.  134.  144.
  232.   81.  104.   59.  246.  297.  258.  229.  275.  281.  179.  200.
  200.  173.  180.   84.  121.  161.   99.  109.  115.  268.  274.  158.
  107.   83.  103.  272.   85.  280.  336.  281.  118.  317.  235.   60.
  174.  259.  178.  128.   96.  126.  288.   88.  292.   71.  197.  186.
   25.   84.   96.  